> **Note:** This notebook is intended to be run on **Google Colab**.

### I. Load & Imports

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!pip install ultralytics

In [ ]:
import zipfile
import os
import glob
import random
import numpy as np
from ultralytics import YOLO
from IPython.display import Image as IPyImage
from PIL import Image as PILImage
from IPython.display import display
import matplotlib.pyplot as plt

### II. Dataset Extraction & Preparation

In [ ]:
# path to the zip dataset
zip_path = "Tire Dataset.v2i.yolov8.zip"

# directory where the dataset will be extracted
extract_path = "/content/dataset"

# create the directory if it does not exist
os.makedirs(extract_path, exist_ok=True)

# extract all files from the zip archive
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction terminée. Les fichiers ont été extraits dans :", extract_path)


In [ ]:
# rename the "valid" folder to "val"
os.rename('/content/dataset/valid', '/content/dataset/val')

# read the content of a label file
with open('/content/dataset/train/labels/1001_png.rf.05300e4f0f2316063d99d0ca39a76ca6.txt') as f:
    lines = f.readlines()
    print("Contenu du fichier :")
    print(lines)

# extract class IDs from the label file
class_ids = [line.strip().split()[0] for line in lines]

# list unique class IDs used
unique_ids = sorted(set(class_ids))
print("\nClasses utilisées dans ce fichier :", unique_ids)


In [ ]:
# read and display the content of the Roboflow README file
with open('/content/dataset/README.roboflow.txt') as f:
    print(f.read())

### III. EDA

In [ ]:
# Base directory containing the YOLO dataset
base = "/content/dataset"

# Dataset splits to analyze
splits = ["train", "val", "test"]

# Loop through each split (train/val/test)
for split in splits:
    # Build the path to the images and labels folders
    img_dir = os.path.join(base, split, "images")
    lbl_dir = os.path.join(base, split, "labels")

    # Count images only if the directory exists
    n_images = len(os.listdir(img_dir)) if os.path.exists(img_dir) else 0

    # Count label files only if the directory exists
    n_labels = len(os.listdir(lbl_dir)) if os.path.exists(lbl_dir) else 0

    # Display a formatted summary for the current split
    print(f"{split.upper()} – Images: {n_images}, Labels: {n_labels}")


In [ ]:
# List to store the resolutions (width, height) of all images
resolutions = []

# Path to the training images
img_dir = "/content/dataset/train/images"

# Loop through all images in the training folder
for img_name in os.listdir(img_dir):
    img_path = os.path.join(img_dir, img_name)

    # Open the image using PIL and retrieve its resolution
    with PILImage.open(img_path) as img:
        resolutions.append(img.size)  # (width, height)

# Display the first 10 resolutions and the number of unique resolutions
resolutions[:10], f"Unique resolutions: {len(set(resolutions))}"


Toutes les images du jeu d’entraînement présentent une résolution unique de 640 × 640 pixels.
Signification:
- uniformisation du dataset avant publication sur Roboflow
- pas de redimensionnement complexe ou de padding supplémentaire
- entraînement plus stable et cohérent, car toutes les images ont exactement les mêmes dimensions

Conclusion :
- résolution homogène
- aucun outlier
- dataset propre et optimisé pour YOLOv8

In [ ]:
label_files = glob.glob("/content/dataset/train/labels/*.txt")

# Dictionary to store how many annotations exist for each class ID
class_counts = {}

# Loop through all label files in the training set
for lf in label_files:
    with open(lf) as f:
        # Each line corresponds to one bounding box
        for line in f:
            cls_id = line.split()[0]          # class ID is the first element of each line
            # Increment the count for this class
            class_counts[cls_id] = class_counts.get(cls_id, 0) + 1

# Display the total number of annotations per class
class_counts


- Le dataset contient 5285 instances de la seule classe “car-tire" (pas de déséquilibre)
- Volume d’annotations suffisant pour entraîner efficacement un modèle léger

In [ ]:
def show_bbox(img_path, label_path):
    # Load the image as a NumPy array
    img = np.array(PILImage.open(img_path))

    # Image height and width
    h, w, _ = img.shape

    # Read YOLO label file (each line contains one bounding box)
    with open(label_path) as f:
        labels = f.readlines()

    # Display the image
    plt.figure(figsize=(6,6))
    plt.imshow(img)

    # Draw each bounding box
    for line in labels:
        cls, x, y, bw, bh = map(float, line.split())

        # Convert YOLO normalized coordinates to pixel values
        x1 = int((x - bw/2) * w)
        y1 = int((y - bh/2) * h)
        x2 = int((x + bw/2) * w)
        y2 = int((y + bh/2) * h)

        # Draw rectangle around detected object
        plt.gca().add_patch(
            plt.Rectangle(
                (x1, y1),         # top-left corner
                x2 - x1,          # width
                y2 - y1,          # height
                edgecolor="red",  # bounding box color
                facecolor="none",
                linewidth=2
            )
        )

    plt.axis("off")
    plt.show()


# Test the function on a random image

img_dir = "/content/dataset/train/images"
label_dir = "/content/dataset/train/labels"

# Pick a random image from the training folder
sample = random.choice(os.listdir(img_dir))

# Display image with its corresponding bounding boxes
show_bbox(
    os.path.join(img_dir, sample),
    os.path.join(label_dir, sample.replace(".jpg", ".txt"))
)


### IV. YAML Configuration File

In [ ]:
# create and write the corrected YAML configuration file
fixed_yaml = """
train: ../train/images
val: ../val/images
test: ../test/images

nc: 1
names: ['car-tire']

roboflow:
  workspace: iotml
  project: tire-dataset
  version: 2
  license: Public Domain
  url: https://universe.roboflow.com/iotml/tire-dataset/dataset/2
"""

# save the YAML file to the dataset directory
with open('/content/dataset/data.yaml', 'w') as f:
    f.write(fixed_yaml)


### V. Model Training & Validation

In [ ]:
# load the YOLOv8 nano model
model = YOLO("yolov8n.pt")

# train on the training set and evaluate on the validation set defined in data.yaml
model.train(data="/content/dataset/data.yaml", epochs=50, imgsz=640)



In [ ]:
# display the confusion matrix image from the training results
IPyImage(filename='runs/detect/train/confusion_matrix.png')

In [ ]:
# display an image with a fixed width
def display_image(path, width=800):
    img = PILImage.open(path)
    display(img.resize((width, int(img.height * width / img.width))))

# display the main training result plots (reduced size)
display_image('runs/detect/train/results.png')
display_image('runs/detect/train/BoxPR_curve.png')
display_image('runs/detect/train/BoxF1_curve.png')
display_image('runs/detect/train/BoxP_curve.png')
display_image('runs/detect/train/BoxR_curve.png')
display_image('runs/detect/train/confusion_matrix.png')
display_image('runs/detect/train/labels.jpg')


- Le modèle YOLOv8n présente d’excellentes performances de détection, avec un mAP50 proche de 95%
- Les métriques de précision et de rappel (~91% chacune) indiquent que le modèle détecte les objets de manière fiable, avec peu de faux positifs et peu de faux négatifs. 
- Le score F1 maximal atteint environ 0.91 à un seuil de confiance optimal d’environ 0.34, ce qui représente le meilleur compromis entre précision et rappel.

Le mAP50-95 (~64%) montre une localisation correcte des bounding boxes, même si elle peut être optimisée davantage en utilisant un modèle plus grand, comme YOLOv8m, pour améliorer la précision des détections aux seuils d’IoU plus stricts.

### VI. Model Testing

In [ ]:
# load the trained YOLO model
trained_model = YOLO("runs/detect/train/weights/best.pt")

# evaluate the trained model using the test split
metrics = trained_model.val(data="/content/dataset/data.yaml", split="test")

- Le modèle obtient de très bonnes performances sur le jeu de test, avec un mAP50 de 92.6% et un F1-score élevé (liés à une précision de 87.6% et un rappel de 89.6%).
- Le modèle détecte les pneus de manière fiable, avec un très bon équilibre entre faux positifs et faux négatifs
- Le mAP50-95 (61.3%) suggère que la localisation des boîtes est correcte mais pourrait être améliorée avec un modèle plus grand comme YOLOv8m.

In [ ]:
# display a validation/test image with a fixed width
def display_image_val(path, width=500):
    img = PILImage.open(path)
    display(img.resize((width, int(img.height * width / img.width))))

# display evaluation performance plots
display_image_val("runs/detect/val/BoxF1_curve.png")
display_image_val("runs/detect/val/BoxPR_curve.png")
display_image_val("runs/detect/val/BoxP_curve.png")
display_image_val("runs/detect/val/BoxR_curve.png")
display_image_val("runs/detect/val/confusion_matrix.png")

# display an example of prediction vs ground truth
display_image_val("runs/detect/val/val_batch0_labels.jpg")  # ground truth
display_image_val("runs/detect/val/val_batch0_pred.jpg")    # model predictions


- F1 ≈ 0.89 : très bon équilibre précision/rappel
- Seuil optimal ≈ 0.17 : le modèle a tendance à être performant même en gardant des prédictions de faible confiance
- mAP50 très élevé (0.926) : le modèle détecte très bien la présence de l’objet
- mAP50-95 plus faible (0.613) 

### VII. Qualitative Evaluation (Visual Comparison)

In [ ]:
# display two images side by side for comparison
def display_side_by_side(img1_path, img2_path, label1="Ground Truth", label2="Prediction"):
    img1 = PILImage.open(img1_path)
    img2 = PILImage.open(img2_path)

    fig, axs = plt.subplots(1, 2, figsize=(14, 7))

    axs[0].imshow(img1)
    axs[0].set_title(label1, fontsize=14)
    axs[0].axis("off")

    axs[1].imshow(img2)
    axs[1].set_title(label2, fontsize=14)
    axs[1].axis("off")

    plt.tight_layout()
    plt.show()

# test with batch 0
display_side_by_side(
    "runs/detect/val/val_batch0_labels.jpg",
    "runs/detect/val/val_batch0_pred.jpg"
)


### VIII. Inference on New Images (Real-World Testing)

In [ ]:
# run inference on an image and save the predictions
results = trained_model.predict(
    source="/content/auto9.jpg",
    conf=0.239,
    save=True
)


In [ ]:
display(PILImage.open("runs/detect/predict/auto1.jpg"))

In [ ]:
results_2 = trained_model.predict(source="/content/auto2.jpg", conf=0.239, save=True)
display(PILImage.open("runs/detect/predict/auto2.jpg"))

In [ ]:
results_3 = trained_model.predict(source="/content/auto3.jpg", conf=0.239, save=True)
display(PILImage.open("runs/detect/predict/auto3.jpg"))

In [ ]:
results_4 = trained_model.predict(source="/content/auto4.jpg", conf=0.239, save=True)
display(PILImage.open("runs/detect/predict/auto4.jpg"))

In [ ]:
results_5 = trained_model.predict(source="/content/auto5.jpg", conf=0.239, save=True)
display(PILImage.open("runs/detect/predict/auto5.jpg"))

In [ ]:
results_6 = trained_model.predict(source="/content/auto6.jpg", conf=0.239, save=True)
display(PILImage.open("runs/detect/predict/auto6.jpg"))

In [ ]:
results_7 = trained_model.predict(source="/content/auto7.jpg", conf=0.239, save=True)
display(PILImage.open("runs/detect/predict/auto7.jpg"))

In [ ]:
results_8 = trained_model.predict(source="/content/auto9.jpg", conf=0.239, save=True)
display(PILImage.open("runs/detect/predict/auto9.jpg"))